In [1]:
import os
import pandas as pd
import numpy as np
from numpy import sort
from os import listdir
from os.path import isfile, join

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import plotly.express as px

from matplotlib_venn import venn3
from Project.Utils.norm import norm
from Project.Utils.max_corr import max_corr

In [2]:
data_path = os.getcwd() + '/Databases/'
output_path = os.getcwd() + '/Output/'
country_path = output_path + '/Country/'
region_path = output_path + '/Region/'
cluster_path = output_path + '/Cluster/'

In [3]:
file_regions = 'AuxiliarData/world-regions-mod.csv'

file_gold = 'GoldDataframe.csv'
file_corr_pearson = 'Corr_DF_Pearson.csv'
file_corr_spearman = 'Corr_DF_Spearman.csv'
file_agg_region = 'AggregatedRegion_DataFrame.csv'
file_agg_world = 'AggregatedWorld_DataFrame.csv'
file_norm_df = 'Norm_DF.csv'
file_shifted_corr_country = 'Shifted_Corr_Country.csv'
file_shifted_corr_region = 'Shifted_Corr_Region.csv'

In [4]:
col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'
col_cluster = 'Cluster'
col_shift = 'Shift'

col_1comp = '1st_component'
col_2comp = '2nd_component'

In [5]:
# READ GOLD DATAFRME
# Read Golden Dataframe and initialize the variables that depend on it.

df = pd.read_csv(output_path + file_gold, on_bad_lines = 'warn', index_col = ['Region', 'Country', 'Year'])

# List of countries and list of regions.
country_list = list(np.sort(df.index.get_level_values(col_country).unique()))
region_list = list(np.sort(df.index.get_level_values(col_region).unique()))

mode_dict = {}
mode_dict[col_country] = country_list
mode_dict[col_region] = region_list
mode_list = [col_country, col_region]

# Range of years.
min_year = df.index.get_level_values(col_year).min()
max_year = df.index.get_level_values(col_year).max()

# List of all indicators, except for the GDP.
indicators_list = df.columns.tolist()
indicators_list.remove(col_gdp)
indicators_list.sort()

In [6]:
# READ CLUSTERS
# Read both Dataframes for each indicators group: the one with the indicator values, and the one with the components.

ind_dict = {}
cluster_dict = {}
cluster_list = []
comp_dict = {}

for element in listdir(cluster_path):
    url = join(cluster_path, element)
    if isfile(url) and url.endswith('_Comp.csv'):
        ind = element.removesuffix('_Comp.csv')
        comp_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        comp_dict[ind] = comp_df
        cluster_number_df = comp_df.drop(columns = comp_df.columns).reset_index(col_cluster, drop = False).rename({col_cluster: ind}, axis = 1)
        cluster_list.append(cluster_number_df)

    elif isfile(url) and url.endswith('.csv'):
        ind = element.removesuffix('.csv')
        ind_df = pd.read_csv(url, on_bad_lines = 'warn', index_col = [col_cluster, col_country])
        cluster_dict[ind] = ind_df.reset_index(col_cluster, drop = False)
        ind_dict[ind] = ind_df.columns

cluster_df = pd.concat(cluster_list, axis = 1)
cluster_df = cluster_df.fillna(-1).astype(int)

In [7]:
cluster_df

,All indicators,Economic Indicators,Equality Indicators,Social-demographic Indicators
Country,,,,
Albania,0,6,4,3
Algeria,0,7,6,9
Armenia,0,1,8,3
Bosnia and Herzegovina,0,1,0,3
Bulgaria,0,2,0,3
...,...,...,...,...
Sao Tome and Principe,9,1,8,5
Solomon Islands,9,6,8,5
Togo,9,4,8,4


In [8]:
class WidgetGlobalStatus:
    def __init__(self):

        self.mode = None #mode_list[0]
        self.zone = None
        self.ind = None
        self.ind_group = None
        self.norm_df = pd.read_csv(output_path + file_norm_df, index_col = [col_country, col_region, col_year])
        self.cluster_df = cluster_df
        self.comp_df = None
        self.ind_df = None
        self.shift_df = None
        self.data = None
        self.years = None #range(min_year, max_year + 1)
        self.shift = None #0

status_widget = WidgetGlobalStatus()

In [9]:
def group_of(ind: str):     # NEEDS ACCESS TO IND_DICT AND A LIST OF ALL INDICATORS
    for group in ind_dict.keys():
        if (len(ind_dict[group]) < len(indicators_list)) and (ind in ind_dict[group]):
            return group
    return None
    return [group if (ind in ind_dict[group]) else None for group in ind_dict.keys()]

# FIND A GOOD WAY TO AVOID RETURNING THE ALL INDICATORS GROUP

In [11]:
plot_cluster_map_out = widgets.Output(layout={'border': '1px solid black'})
table_cluster_out = widgets.Output(layout={'border': '1px solid black'})
venn_group_out = widgets.Output(layout={'border': '1px solid black'})
chart_shift_ind_out = widgets.Output(layout={'border': '1px solid black'})
chart_shift_corr_out = widgets.Output(layout={'border': '1px solid black'})

# CLUSTER MAP

In [12]:
plot_cluster_map_out

Output(layout=Layout(border='1px solid black'))

# CLUSTER TABLE

In [13]:
table_cluster_out

Output(layout=Layout(border='1px solid black'))

# CLUSTER VENN DIAGRAM

In [14]:
venn_group_out

Output(layout=Layout(border='1px solid black'))

# CHART OF INDICATORS VALUES

In [15]:
chart_shift_ind_out

Output(layout=Layout(border='1px solid black'))

# CHART OF INDICATORS CORRELATIONS

In [16]:
chart_shift_corr_out

Output(layout=Layout(border='1px solid black'))

In [17]:
@plot_cluster_map_out.capture(clear_output=True, wait=True)
def plot_cluster_map (df: pd.DataFrame, group_name: str = 'All indicators', *, col_1comp: str = '1st_component', col_2comp: str = '2nd_component'):
    if df is None: return None
    fig = px.scatter(df, x = col_1comp, y = col_2comp, text = df.index, size_max=100, color=col_cluster, category_orders={col_cluster: np.sort(df.loc[:, col_cluster].unique())})
    fig.update_layout(title_text = group_name, title_x=0.5)
    fig.update_traces(textposition = 'top center')
    fig.show()

@table_cluster_out.capture(clear_output=True, wait=True)
def table_cluster (df: pd.DataFrame, group_name: str = 'All indicators', country: str = '', cluster_number: int = 0):
    if df is None: return None
    try:
        df_s = df.style
        df_s.apply_index(lambda i: ['background-color: #aadfff; font-weight: 500' if c == country else '' for c in i], axis = 0)
        df_s.apply(lambda row: ['background-color: #ccebff;' if row.name == country else '' for cell in row], axis = 1)
        df_s.set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#ddfdff')]}])
        tt = {}
        for col in df.columns:
            tt[col] = 'Column median: ' + str(df.loc[:, col].median())
        df_s.set_tooltips(pd.DataFrame(tt, index = df.index))

        # Display a short descriptive title and the Dataframe.
        display(country + ' belongs to Cluster ' + str(cluster_number) + '. This Cluster contains a total of ' + str(df.shape[0]) + ' countries.')
        display(df_s)
    except Exception:
        return print('No indicators available for this country.')

@venn_group_out.capture(clear_output=True, wait=True)
def venn_group(cluster_df, country):
    try:
        _, econ_ind, eq_ind, socdem_ind = ind_dict.keys()       # NON ROBUST STATEMENT
        _, set_econ, set_eq, set_socdem = (set(cluster_df.loc[lambda df: df[ind] == df.loc[country, ind], ind].index.to_list()) for ind in ind_dict.keys())
        venn = venn3([set_econ, set_socdem, set_eq], (econ_ind, socdem_ind, eq_ind))

        venn.get_label_by_id('100').set_text('\n'.join(set_econ - set_socdem - set_eq)) # Only econ
        venn.get_label_by_id('010').set_text('\n'.join(set_socdem - set_econ - set_eq)) # Only socdem
        venn.get_label_by_id('001').set_text('\n'.join(set_eq - set_econ - set_socdem)) # Only eq

        # The three pair-intersections is guaranteed only if there is an intersection of the three groups.
        if len(set_econ & set_socdem & set_eq):
            venn.get_label_by_id('111').set_text('\n'.join(set_econ & set_socdem & set_eq))
            venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))
        else:
            # If no center, check the intersections that do exist.
            if len(set_econ & set_socdem - set_eq):
                venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            if len(set_econ & set_eq - set_socdem):
                venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            if len(set_socdem & set_eq - set_econ):
                venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))

        plt.rcParams["figure.figsize"] = (12, 12)
        plt.show()

    except:
        return print('No indicators available for this country.')

@chart_shift_ind_out.capture(clear_output=True, wait=True)
def chart_shift_ind(norm_df, mode, zone, indicator, shift):
    data_s = norm_df.loc[norm_df.index.get_level_values(mode) == zone, [col_gdp, indicator]].groupby(level = col_year).median()

    min_year_gdp = min_year + max(shift, 0)
    max_year_gdp = max_year + min(shift, 0)

    min_year_ind = min_year - min(shift, 0)
    max_year_ind = max_year - max(shift, 0)

    norm_gdp = norm(data_s.loc[min_year_gdp : max_year_gdp, [col_gdp]], None)
    norm_ind = norm(data_s.loc[min_year_ind : max_year_ind, [indicator]], None)

    plt.figure(figsize = (8,8))
    plt.plot(
                #norm_gdp.index.get_level_values(col_year),
                norm_gdp.reset_index(drop = True),
                color = "red", label = col_gdp)
    plt.plot(
                #norm_ind.index.get_level_values(col_year),
                norm_ind.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")
    plt.show()

@chart_shift_corr_out.capture(clear_output=True, wait=True)
def chart_shift_corr(mode, zone, indicator):
    df_m = pd.read_csv(output_path + 'Shifted_Corr_' + mode + '.csv', index_col = [mode, col_shift])
    df_s = df_m.loc[df_m.index.get_level_values(mode) == zone, indicator]

    plt.figure(figsize = (8,8))
    plt.plot(
                df_s.index.get_level_values(col_shift),
                df_s.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")
    plt.show()

# WIDGETS

In [18]:
dropdown_mode = widgets.Dropdown(
    options = mode_list,
    description = 'Select: '
)

dropdown_zone = widgets.Dropdown(
    options = mode_dict[dropdown_mode.value],
    description = 'Show: '
)

dropdown_ind = widgets.Dropdown(
    options = indicators_list,
    description = 'Indicator: '
)

intslider_years = widgets.IntRangeSlider(
    value = [min_year, max_year],
    min = min_year,
    max = max_year,
    step = 1,
    description = 'Years: ',
)

intslider_shift = widgets.IntSlider(
    value = 0,
    min = min_year - max_year + 1,
    max = max_year - min_year - 1,
    step = 1,
    description = 'Shifts: '
)

In [19]:
def updatePlots(mode, zone, ind, years, shift):
    if ind != status_widget.ind:
        if mode == 'Region':
            status_widget.ind = None
            status_widget.ind_group = None
        else:
            status_widget.ind = ind
            group = group_of(ind)
            if status_widget.ind_group != group: status_widget.ind_group = group
    if mode != status_widget.mode:
        dropdown_zone.options = mode_dict[mode]
        status_widget.mode = mode
        data = None
    
    if zone != status_widget.zone:
        if mode == col_country:
            #print(status_widget.ind_group)
            #display(comp_dict[status_widget.ind_group])
            status_widget.comp_df = comp_dict[status_widget.ind_group].reset_index(col_cluster, drop = False)
        data = None
        status_widget.zone = zone
    
    if shift != status_widget.shift:
        status_widget.shift = shift
    
    cluster_number = cluster_dict[status_widget.ind_group].loc[cluster_dict[status_widget.ind_group].index == status_widget.zone, col_cluster].item()
    df_ind = cluster_dict[status_widget.ind_group]
    status_widget.ind_df = df_ind.loc[df_ind[col_cluster] == cluster_number].drop(col_cluster, axis = 'columns')

    # DO THE ILLEGAL THING
    plot_cluster_map (status_widget.comp_df)
    table_cluster (status_widget.ind_df, status_widget.ind_group, status_widget.zone)
    venn_group (status_widget.cluster_df, status_widget.zone)

    chart_shift_ind(status_widget.norm_df, status_widget.mode, status_widget.zone, status_widget.ind, status_widget.shift)
    chart_shift_corr(status_widget.mode, status_widget.zone, status_widget.ind)

widgets.interact(updatePlots, mode = dropdown_mode, zone = dropdown_zone, ind = dropdown_ind,
                 years = intslider_years, shift = intslider_shift)

interactive(children=(Dropdown(description='Select: ', options=('Country', 'Region'), value='Country'), Dropdo…

<function __main__.updatePlots(mode, zone, ind, years, shift)>

# MY DASHBOARD

In [20]:
ind_dict['All indicators']
np.sort(cluster_df.loc[:, 'All indicators'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [21]:
""" def cluster_map (indicators):
    comp_df = comp_dict[indicators].reset_index(col_cluster, drop = False)
    
    # Show the resulting chart
    fig = px.scatter(comp_df, x = col_1comp, y = col_2comp, text = comp_df.index, size_max=100, color=col_cluster, category_orders={col_cluster: sort(list(set(comp_df.loc[:, col_cluster])))})
    fig.update_layout(title_text = indicators, title_x=0.5)
    fig.update_traces(textposition = 'top center')
    fig.show()

@interact(
    indicators = cluster_dict.keys(),
)

def g(indicators):
    return cluster_map (indicators)   
 """

" def cluster_map (indicators):\n    comp_df = comp_dict[indicators].reset_index(col_cluster, drop = False)\n    \n    # Show the resulting chart\n    fig = px.scatter(comp_df, x = col_1comp, y = col_2comp, text = comp_df.index, size_max=100, color=col_cluster, category_orders={col_cluster: sort(list(set(comp_df.loc[:, col_cluster])))})\n    fig.update_layout(title_text = indicators, title_x=0.5)\n    fig.update_traces(textposition = 'top center')\n    fig.show()\n\n@interact(\n    indicators = cluster_dict.keys(),\n)\n\ndef g(indicators):\n    return cluster_map (indicators)   \n "

In [22]:
""" def tableCountry(Ind, Country):
    try:
        # Find the cluster Country belongs to.
        cluster_number = cluster_dict[Ind].loc[cluster_dict[Ind].index == Country, col_cluster].item()
        # Retrieve the Dataframe with the selected indicators, and filter to only show the rows (countries) belonging to the cluster. Drop the cluster number column as it is redundant.
        df_ind = cluster_dict[Ind]
        df = df_ind.loc[df_ind[col_cluster] == cluster_number].drop(col_cluster, axis = 'columns')
        # Format the Dataframe representation.
        df_s = df.style
        df_s.apply_index(lambda i: ['background-color: #aadfff; font-weight: 500' if c == Country else '' for c in i], axis = 0)
        df_s.apply(lambda row: ['background-color: #ccebff;' if row.name == Country else '' for cell in row], axis = 1)
        df_s.set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#ddfdff')]}])
        tt = {}
        for col in df.columns:
            tt[col] = 'Column median: ' + str(df.loc[:, col].median())
        df_s.set_tooltips(pd.DataFrame(tt, index = df.index))

        # Display a short descriptive title and the Dataframe.
        display(Country + ' belongs to Cluster ' + str(cluster_number) + '. This Cluster contains a total of ' + str(df.shape[0]) + ' countries.')
        display(df_s)
    except Exception:
        return print('No indicators available for this country.')

@interact(
    Indicators = cluster_dict.keys(),
    Country = country_list
)

def g(Indicators = 'Equality indicators', Country = 'Afghanistan'):
    return tableCountry(Indicators, Country)     """

" def tableCountry(Ind, Country):\n    try:\n        # Find the cluster Country belongs to.\n        cluster_number = cluster_dict[Ind].loc[cluster_dict[Ind].index == Country, col_cluster].item()\n        # Retrieve the Dataframe with the selected indicators, and filter to only show the rows (countries) belonging to the cluster. Drop the cluster number column as it is redundant.\n        df_ind = cluster_dict[Ind]\n        df = df_ind.loc[df_ind[col_cluster] == cluster_number].drop(col_cluster, axis = 'columns')\n        # Format the Dataframe representation.\n        df_s = df.style\n        df_s.apply_index(lambda i: ['background-color: #aadfff; font-weight: 500' if c == Country else '' for c in i], axis = 0)\n        df_s.apply(lambda row: ['background-color: #ccebff;' if row.name == Country else '' for cell in row], axis = 1)\n        df_s.set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#ddfdff')]}])\n        tt = {}\n        for col in df.columns:\n     

In [23]:
""" def VennOut(Country):
    try:
        _, econ_ind, eq_ind, socdem_ind = ind_dict.keys()       # NON ROBUST STATEMENT
        _, set_econ, set_eq, set_socdem = (set(cluster_df.loc[lambda df: df[ind] == df.loc[Country, ind], ind].index.to_list()) for ind in ind_dict.keys())
        venn = venn3([set_econ, set_socdem, set_eq], (econ_ind, socdem_ind, eq_ind))

        venn.get_label_by_id('100').set_text('\n'.join(set_econ - set_socdem - set_eq)) # Only econ
        venn.get_label_by_id('010').set_text('\n'.join(set_socdem - set_econ - set_eq)) # Only socdem
        venn.get_label_by_id('001').set_text('\n'.join(set_eq - set_econ - set_socdem)) # Only eq

        # The three pair-intersections is guaranteed only if there is an intersection of the three groups.
        if len(set_econ & set_socdem & set_eq):
            venn.get_label_by_id('111').set_text('\n'.join(set_econ & set_socdem & set_eq))
            venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))
        else:
            # If no center, check the intersections that do exist.
            if len(set_econ & set_socdem - set_eq):
                venn.get_label_by_id('110').set_text('\n'.join(set_econ & set_socdem - set_eq))
            if len(set_econ & set_eq - set_socdem):
                venn.get_label_by_id('101').set_text('\n'.join(set_econ & set_eq - set_socdem))
            if len(set_socdem & set_eq - set_econ):
                venn.get_label_by_id('011').set_text('\n'.join(set_socdem & set_eq - set_econ))

        plt.rcParams["figure.figsize"] = (12, 12)
        plt.show()

    except:
        return print('No indicators available for this country.')

@interact(
    Country = country_list)

def g(Country = 'Spain'):
    return VennOut(Country) """

' def VennOut(Country):\n    try:\n        _, econ_ind, eq_ind, socdem_ind = ind_dict.keys()       # NON ROBUST STATEMENT\n        _, set_econ, set_eq, set_socdem = (set(cluster_df.loc[lambda df: df[ind] == df.loc[Country, ind], ind].index.to_list()) for ind in ind_dict.keys())\n        venn = venn3([set_econ, set_socdem, set_eq], (econ_ind, socdem_ind, eq_ind))\n\n        venn.get_label_by_id(\'100\').set_text(\'\n\'.join(set_econ - set_socdem - set_eq)) # Only econ\n        venn.get_label_by_id(\'010\').set_text(\'\n\'.join(set_socdem - set_econ - set_eq)) # Only socdem\n        venn.get_label_by_id(\'001\').set_text(\'\n\'.join(set_eq - set_econ - set_socdem)) # Only eq\n\n        # The three pair-intersections is guaranteed only if there is an intersection of the three groups.\n        if len(set_econ & set_socdem & set_eq):\n            venn.get_label_by_id(\'111\').set_text(\'\n\'.join(set_econ & set_socdem & set_eq))\n            venn.get_label_by_id(\'110\').set_text(\'\n\'.joi

In [24]:
norm_df = pd.read_csv(output_path + 'Norm_DF.csv', index_col = [col_country, col_region, col_year])

In [25]:
""" # Widgets: dropdowns for country or area, select a particular country or area and select an indicator; and an intslider to choose the shift to apply.

dropdown_select = widgets.Dropdown(
    options = mode_list,
    description = 'Select: '
)

dropdown_show_e = widgets.Dropdown(
    options = mode_dict[dropdown_select.value],
    description = 'Show: '
)

dropdown_indicators = widgets.Dropdown(
    options = indicators_list,
    description = 'Indicator: '
)

intslider_shift = widgets.IntSlider(
    value = 0,
    min = min_year - max_year + 1,
    max = max_year - min_year - 1,
    step = 1,
    description = 'Shifts: ')

def change_selection (selector):
    dropdown_show_e.options = mode_dict[selector]

widgets.interact(change_selection, selector = dropdown_select)

def show_chart(element, indicator, shift):
    data_s = norm_df.loc[norm_df.index.get_level_values(dropdown_select.value) == element, [col_gdp, indicator]].groupby(level = col_year).median()

    min_year_gdp = min_year + max(shift, 0)
    max_year_gdp = max_year + min(shift, 0)

    min_year_ind = min_year - min(shift, 0)
    max_year_ind = max_year - max(shift, 0)

    norm_gdp = norm(data_s.loc[min_year_gdp : max_year_gdp, [col_gdp]], None)
    norm_ind = norm(data_s.loc[min_year_ind : max_year_ind, [indicator]], None)

    plt.figure(figsize = (8,8))
    plt.plot(
                #norm_gdp.index.get_level_values(col_year),
                norm_gdp.reset_index(drop = True),
                color = "red", label = col_gdp)
    plt.plot(
                #norm_ind.index.get_level_values(col_year),
                norm_ind.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")

widgets.interact(show_chart, element = dropdown_show_e, indicator = dropdown_indicators, shift = intslider_shift) """

' # Widgets: dropdowns for country or area, select a particular country or area and select an indicator; and an intslider to choose the shift to apply.\n\ndropdown_select = widgets.Dropdown(\n    options = mode_list,\n    description = \'Select: \'\n)\n\ndropdown_show_e = widgets.Dropdown(\n    options = mode_dict[dropdown_select.value],\n    description = \'Show: \'\n)\n\ndropdown_indicators = widgets.Dropdown(\n    options = indicators_list,\n    description = \'Indicator: \'\n)\n\nintslider_shift = widgets.IntSlider(\n    value = 0,\n    min = min_year - max_year + 1,\n    max = max_year - min_year - 1,\n    step = 1,\n    description = \'Shifts: \')\n\ndef change_selection (selector):\n    dropdown_show_e.options = mode_dict[selector]\n\nwidgets.interact(change_selection, selector = dropdown_select)\n\ndef show_chart(element, indicator, shift):\n    data_s = norm_df.loc[norm_df.index.get_level_values(dropdown_select.value) == element, [col_gdp, indicator]].groupby(level = col_year)

In [26]:
""" dropdown_select_corr = widgets.Dropdown(
    options = mode_list,
    description = 'Select: '
)

dropdown_show_corr = widgets.Dropdown(
    options = mode_dict[dropdown_select.value],
    description = 'Show: '
)

dropdown_indicators_corr = widgets.Dropdown(
    options = indicators_list,
    description = 'Indicator: '
)

def change_selection_corr (selector):
    dropdown_show_corr.options = mode_dict[selector]

def show_evolution(element, indicator):
    
    df_m = pd.read_csv(output_path + 'Shifted_Corr_' + dropdown_select_corr.value + '.csv', index_col = [col_country, col_shift])
    df_s = df_m.loc[df_m.index.get_level_values(dropdown_select_corr.value) == element, indicator]

    plt.figure(figsize = (8,8))
    plt.plot(
                df_s.index.get_level_values(col_shift),
                df_s.reset_index(drop = True),
                color = "green", label = indicator)
    plt.legend(loc = "lower right")

widgets.interact(change_selection_corr, selector = dropdown_select)

widgets.interact(show_evolution, element = dropdown_show_corr, indicator = dropdown_indicators_corr) """

' dropdown_select_corr = widgets.Dropdown(\n    options = mode_list,\n    description = \'Select: \'\n)\n\ndropdown_show_corr = widgets.Dropdown(\n    options = mode_dict[dropdown_select.value],\n    description = \'Show: \'\n)\n\ndropdown_indicators_corr = widgets.Dropdown(\n    options = indicators_list,\n    description = \'Indicator: \'\n)\n\ndef change_selection_corr (selector):\n    dropdown_show_corr.options = mode_dict[selector]\n\ndef show_evolution(element, indicator):\n    \n    df_m = pd.read_csv(output_path + \'Shifted_Corr_\' + dropdown_select_corr.value + \'.csv\', index_col = [col_country, col_shift])\n    df_s = df_m.loc[df_m.index.get_level_values(dropdown_select_corr.value) == element, indicator]\n\n    plt.figure(figsize = (8,8))\n    plt.plot(\n                df_s.index.get_level_values(col_shift),\n                df_s.reset_index(drop = True),\n                color = "green", label = indicator)\n    plt.legend(loc = "lower right")\n\nwidgets.interact(change_sel